# OpenAI Executing Multiple Functions in a Single Message

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph python-dotenv

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Setup logging
import logging

logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)


In [3]:
from openai_config import OpenAIConfig
# We read the .env file inside the class
openai_config = OpenAIConfig()
openai_config.chat_model.invoke("Tell me a Joke about corporate life, not the ladder one!")

2024-09-17 17:38:09,935 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Why did the employee bring a ladder to the meeting?\n\nBecause they heard the agenda was about "climbing to new heights!" \n\n(Just kidding! Here’s a different one:) \n\nWhy did the scarecrow become a successful corporate manager?\n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 20, 'total_tokens': 76, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-db6e0043-342e-4ffb-83c2-b5aa69ca92d1-0', usage_metadata={'input_tokens': 20, 'output_tokens': 56, 'total_tokens': 76})

In [4]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)



In [5]:
# Demonstrate OpenAI doing function calling
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = llm_with_tools.invoke(messages)
messages.tool_calls

2024-09-17 17:39:24,783 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'name': 'add',
  'args': {'a': 3, 'b': 4},
  'id': 'call_Q0edODpkAMJuaeazfQLLPuVA',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 7, 'b': 2},
  'id': 'call_OKVtOE0FavLy7kkh8RuKD3zo',
  'type': 'tool_call'},
 {'name': 'divide',
  'args': {'a': 14, 'b': 5},
  'id': 'call_WtG1OruEplv7TGizRTqcSe6k',
  'type': 'tool_call'}]